## Results

1. We have successfully reach 90% accuracy on validation set using resnet50 in 801 epochs

![alt text](resnet50_epoch_801.png "ResNet50")

In [1]:
import torch
import torch.nn as nn
from models.resnet import resnet50  # Adjust based on your specific ResNet model (resnet34 or resnet50)
from utils.dataset import get_CIFAR_10  # Function to load CIFAR-10 data
from utils.evaluation import evaluate_model  # Import your evaluation function
from utils.training import load_checkpoint
import torch.optim as optim
# Set device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load CIFAR-10 dataset (using validation loader)
_, val_loader = get_CIFAR_10(batch_size=256, num_workers=8, augmentation='resnet')  # Load validation data

# Initialize the ResNet model (same model architecture as the one used for training)
model = resnet50(num_classes=10).to(device)  # Adjust if you trained a different ResNet model like resnet50

# Load the trained model weights
# model.load_state_dict(torch.load('resnet18_model.pth'))
# model.to(device)
# print("Model loaded successfully from 'resnet_model.pth'.")
# Assuming `model` and `optimizer` are already defined
optimizer = optim.SGD(model.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
model, optimizer, epoch, best_val_accuracy = load_checkpoint('resnet50_model.pth', model, optimizer)
# model.load_state_dict(torch.load('resnet50_model.pth'))
# Define the loss function (same as used during training)
criterion = nn.CrossEntropyLoss()

# Evaluate the loaded model on the validation set
evaluation_results = evaluate_model(model, val_loader, criterion, device)

# Print the evaluation results
print(f"Validation Loss: {evaluation_results['loss']:.4f}")
print(f"Validation Accuracy: {evaluation_results['accuracy'] * 100:.2f}%")

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified
Checkpoint loaded: Epoch 19, Best Validation Accuracy: 0.9108
Evaluation - Loss: 0.2880, Accuracy: 0.9108
Validation Loss: 0.2880
Validation Accuracy: 91.08%


2. I failed the implementation of MobiNet, I did not see a complete description how the architect was designed and any implementation online. Here is the result, I switch to implementing Mobile Net as I am not really sure which one I should do. The paper is attached here: 

MoBiNet: A Mobile Binary Network for Image Classification

### 3. Mobile Net V1

Successfully replicate teh Mobile Net V1 following the paper and guidance online. Reach 80.73% accuracy in 80 epochs. However the model reaches its limit around 380 epochs as the training accuracy reaches 99.77% with best validation accuracy only 85.23%. This shows that the model encountered overffiting. After adding drop out layer before fully connected layer, we reach validation accuracy 89.67%. I am still trying to improve this result.

![alt text](MobileNetV1(ndp)_epoch_380.png "MobileNetV1")


In [1]:
import torch
import torch.nn as nn
from models.MobileNetV1 import MobileNetV1, mobilenet_v1 # Adjust based on your specific ResNet model (resnet34 or resnet50)
from utils.dataset import get_CIFAR_10  # Function to load CIFAR-10 data
from utils.evaluation import evaluate_model  # Import your evaluation function
from utils.training import load_checkpoint
import torch.optim as optim
# Set device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load CIFAR-10 dataset (using validation loader)
_, val_loader = get_CIFAR_10(batch_size=256, num_workers=8, augmentation='mobilenetv1')  # Load validation data

# Initialize MobileNetV1 model with width and resolution multipliers
model = mobilenet_v1(num_classes=10, width_multiplier=0.75, resolution_multiplier=0.5).to(device)

# Load the trained model weights
# model.load_state_dict(torch.load('resnet18_model.pth'))
# model.to(device)
# print("Model loaded successfully from 'resnet_model.pth'.")
# Assuming `model` and `optimizer` are already defined
optimizer = optim.SGD(model.parameters(), lr=1e-4,
                      momentum=0.9, weight_decay=5e-4)
model, optimizer, epoch, best_val_accuracy = load_checkpoint('mobilenet_v1_model.pth', model, optimizer)
# model.load_state_dict(torch.load('resnet50_model.pth'))
# Define the loss function (same as used during training)
criterion = nn.CrossEntropyLoss()

# Evaluate the loaded model on the validation set
evaluation_results = evaluate_model(model, val_loader, criterion, device)

# Print the evaluation results
print(f"Validation Loss: {evaluation_results['loss']:.4f}")
print(f"Validation Accuracy: {evaluation_results['accuracy'] * 100:.2f}%")

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified
Checkpoint loaded: Epoch 19, Best Validation Accuracy: 0.8345
New Learning Rate: 0.005
Evaluation - Loss: 2.4294, Accuracy: 0.1000
Validation Loss: 2.4294
Validation Accuracy: 10.00%


In [1]:
import torch
import torch.nn as nn
from models.MobileNetV2 import MobileNetV2, mobilenet_v2 # Adjust based on your specific ResNet model (resnet34 or resnet50)
from utils.dataset import get_CIFAR_10  # Function to load CIFAR-10 data
from utils.evaluation import evaluate_model  # Import your evaluation function
from utils.training import load_checkpoint
import torch.optim as optim
# Set device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load CIFAR-10 dataset (using validation loader)
_, val_loader = get_CIFAR_10(batch_size=256, num_workers=8, augmentation='mobilenetv1')  # Load validation data

# Initialize MobileNetV1 model with width and resolution multipliers
# Set device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load CIFAR-10 dataset with data augmentation suitable for MobileNet
train_loader, val_loader = get_CIFAR_10(batch_size=256, num_workers=8, augmentation='resnet')

# Initialize MobileNetV2 model with width and resolution multipliers
model = mobilenet_v2(num_classes=10, width_multiplier=1.0, resolution_multiplier=1.0, dropout_rate=0.2).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-3)

# Define learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
model, optimizer, epoch, best_val_accuracy = load_checkpoint('mobilenet_v2_model.pth', model, optimizer)
# Evaluate the loaded model on the validation set
evaluation_results = evaluate_model(model, val_loader, criterion, device)

# Print the evaluation results
print(f"Validation Loss: {evaluation_results['loss']:.4f}")
print(f"Validation Accuracy: {evaluation_results['accuracy'] * 100:.2f}%")

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified
Using device: cuda
Files already downloaded and verified
Files already downloaded and verified
Checkpoint loaded: Epoch 19, Best Validation Accuracy: 0.8932


/home/coder/miniconda3/envs/ml/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Evaluation - Loss: 0.3786, Accuracy: 0.8932
Validation Loss: 0.3786
Validation Accuracy: 89.32%
